In [1]:
import sys
import os
sys.path.append(os.path.abspath('../'))
import json
from datetime import datetime,timedelta
import dateutil.parser
import dateutil.rrule
import numpy as np
import pandas as pd
import xarray as xr
from metgrs import Utils
import io
import math
import types
import glob
import matplotlib as mpl
from joblib import Parallel,delayed
from metgrs import WindProfileRadar
import matplotlib.pyplot as plt

In [2]:
datapath='../datas/'

In [3]:
fs=glob.glob(datapath+'/WindProfileRadarFiles/Z_RADA_I_57461_*_R_WPRD_LC_STA.XML')

In [4]:
fs

['../datas//WindProfileRadarFiles\\Z_RADA_I_57461_20241204183600_R_WPRD_LC_STA.XML',
 '../datas//WindProfileRadarFiles\\Z_RADA_I_57461_20241204195400_R_WPRD_LC_STA.XML',
 '../datas//WindProfileRadarFiles\\Z_RADA_I_57461_20241204200000_R_WPRD_LC_STA.XML',
 '../datas//WindProfileRadarFiles\\Z_RADA_I_57461_20241204200600_R_WPRD_LC_STA.XML',
 '../datas//WindProfileRadarFiles\\Z_RADA_I_57461_20241204201200_R_WPRD_LC_STA.XML',
 '../datas//WindProfileRadarFiles\\Z_RADA_I_57461_20241204201800_R_WPRD_LC_STA.XML',
 '../datas//WindProfileRadarFiles\\Z_RADA_I_57461_20241204202400_R_WPRD_LC_STA.XML',
 '../datas//WindProfileRadarFiles\\Z_RADA_I_57461_20241204203000_R_WPRD_LC_STA.XML',
 '../datas//WindProfileRadarFiles\\Z_RADA_I_57461_20241204203600_R_WPRD_LC_STA.XML',
 '../datas//WindProfileRadarFiles\\Z_RADA_I_57461_20241204204200_R_WPRD_LC_STA.XML',
 '../datas//WindProfileRadarFiles\\Z_RADA_I_57461_20241204204800_R_WPRD_LC_STA.XML',
 '../datas//WindProfileRadarFiles\\Z_RADA_I_57461_20241204205400_

In [5]:
ds=list(map(WindProfileRadar.readStatuXMLfile,fs))

In [6]:
ds[0]

{
    "StaticParameters": {'Points': [{'ParameterUnit': '', 'ParameterValue': 'CHIN', 'ParameterName': 'Country'}, {'ParameterUnit': '', 'ParameterValue': 'hubei', 'ParameterName': 'Province'}, {'ParameterUnit': '', 'ParameterValue': '57461', 'ParameterName': 'StationNumber'}, {'ParameterUnit': '', 'ParameterValue': 'yichang', 'ParameterName': 'Station'}, {'ParameterUnit': '', 'ParameterValue': 'LC', 'ParameterName': 'RadarType'}, {'ParameterUnit': '度', 'ParameterValue': 'E111/21/33', 'ParameterName': 'Longitude'}, {'ParameterUnit': '度', 'ParameterValue': 'N030/44/12', 'ParameterName': 'Latitude'}, {'ParameterUnit': 'm', 'ParameterValue': '253.80', 'ParameterName': 'Altitude'}]},
    "SystemStatus": {'Points': [{'ParameterUnit': '', 'ParameterValue': '2024-12-04 18:36:00', 'ParameterName': 'Time'}, {'ParameterUnit': '', 'ParameterValue': '2', 'ParameterName': 'Radarstatus'}, {'ParameterUnit': '', 'ParameterValue': '8', 'ParameterName': 'SubsysNum'}, {'ParameterUnit': '', 'ParameterValu